# 0. Requirements

In [1]:
#nthreads to ran xgboost
nthread = 39

# 1. Load data and libs

In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/ubuntu/xgboost/wrapper/')
import xgboost as xgb

In [3]:
train = pd.read_csv("data/training.csv")
test  = pd.read_csv("data/test.csv")

from sklearn.cross_validation import KFold
kf = KFold(len(train), n_folds=5, random_state=555, shuffle=True)

In [4]:
check_agreement = pd.read_csv('data/check_agreement.csv')

def check_ks(agreement_probs, check_agreement = check_agreement):

    ks = evaluation.compute_ks(
        agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    return 'KS metric', ks, ks < 0.09

check_correlation = pd.read_csv('data/check_correlation.csv', index_col='id')
#evaluation.compute_cvm(correlation_probs, check_correlation['mass'])

In [5]:
# make some physical features

for tt in train,test, check_agreement, check_correlation:
    tt['p0_pz'] = (tt.p0_p**2 - tt.p0_pt**2)**0.5
    tt['p1_pz'] = (tt.p1_p**2 - tt.p1_pt**2)**0.5
    tt['p2_pz'] = (tt.p2_p**2 - tt.p2_pt**2)**0.5
    tt['pz'] = tt.p0_pz + tt.p1_pz + tt.p2_pz
    tt['p'] = (tt.pt**2 + tt.pz**2)**0.5
    tt['speed'] = tt.FlightDistance / tt.LifeTime
    tt['new_mass'] = tt.p / tt.speed

In [6]:
features = list(train.columns[1:47])
features_remove = ['SPDhits']
features_s = [x for x in features if x not in features_remove]

# 2. First part of the solution

In [7]:
%%time

params = {"objective": "binary:logistic",
          "eta": 0.39,
          "max_depth": 100,
          "min_child_weight": 0.1,
          "silent": 1,
          "subsample": 1,
          "colsample_bytree": 0.4,
          'eval_metric': 'auc',
          "seed": 4,
          'nthread': 1}
num_trees=5

features_m = features + ['new_mass']

#cv part
XGB1_train = np.zeros(len(train))

for itrain, itest in kf:
    X_train, y_train = train.loc[itrain, features_m], train.loc[itrain, 'signal']
    X_test, y_test = train.loc[itest, features_m], train.loc[itest, 'signal']
    dtrain = xgb.DMatrix(X_train.values, y_train)
    dtest = xgb.DMatrix(X_test.values, y_test)
    
    gbm_good2 = xgb.train(params, dtrain, num_trees)
    XGB1_train[itest] = gbm_good2.predict(dtest)

#checks & test part
X_train, y_train = train[features_m], train['signal']
gbm_good2 = xgb.train(params, xgb.DMatrix(X_train.values, y_train), num_trees)

XGB1_agreement = gbm_good2.predict(xgb.DMatrix(check_agreement[features_m].values))
XGB1_correlation = gbm_good2.predict(xgb.DMatrix(check_correlation[features_m].values))
XGB1 = gbm_good2.predict(xgb.DMatrix(test[features_m].values))

CPU times: user 20.1 s, sys: 784 ms, total: 20.9 s
Wall time: 20.9 s


In [8]:
import evaluation
from evaluation import compute_cvm
from sklearn.metrics import roc_auc_score

print roc_auc_score(train.signal, XGB1_train)
print check_ks(XGB1_agreement)
print compute_cvm(XGB1_correlation, check_correlation['mass'])

0.984971190223
('KS metric', 0.062314205779322618, True)
0.0317784631198


In [11]:
%%time
params = {"objective": "binary:logistic",
          "eta": 0.2,
          "max_depth": 15,
          "min_child_weight": 15,
          "silent": 1,
          "subsample": 1,
          "colsample_bytree": 0.9,
          'eval_metric': 'auc',
          "seed": 5,
          'nthread': 1}
num_trees=35

#cv part
XGB2_train = np.zeros(len(train))

for itrain, itest in kf:
    X_train, y_train = train.loc[itrain, features_s], train.loc[itrain, 'signal']
    X_test, y_test = train.loc[itest, features_s], train.loc[itest, 'signal']
    dtrain = xgb.DMatrix(X_train.values, y_train)
    dtest = xgb.DMatrix(X_test.values, y_test)
    
    gbm_bad2 = xgb.train(params, dtrain, num_trees)
    XGB2_train[itest] = gbm_bad2.predict(dtest)

#checks & test part

X_train, y_train = train[features_s], train['signal']
gbm_bad2 = xgb.train(params, xgb.DMatrix(X_train.values, y_train), num_trees)

XGB2_agreement = gbm_bad2.predict(xgb.DMatrix(check_agreement[features_s].values))
XGB2_correlation = gbm_bad2.predict(xgb.DMatrix(check_correlation[features_s].values))
XGB2 = gbm_bad2.predict(xgb.DMatrix(test[features_s].values))

CPU times: user 2min 13s, sys: 591 ms, total: 2min 13s
Wall time: 2min 13s


In [21]:
print roc_auc_score(train.signal, XGB2_train)
print check_ks(XGB2_agreement)
print compute_cvm(XGB2_correlation, check_correlation['mass'])

0.938783611211
('KS metric', 0.095255149957842178, False)
0.000967539980608


# 3. Second part of the solution

In [13]:
%%time

predXGB2_test = []

params = {"objective": "binary:logistic",
          "eta": 0.05,
          "max_depth": 6,
          "min_child_weight": 10,
          "silent": 1,
          "subsample": 1,
          "colsample_bytree": 1,
          'eval_metric': 'auc',
          "seed": 1,
          "nthread": nthread}
num_trees=700

#cv
XGB3_train = np.zeros(len(train))

for train_index, test_index in kf:
    X_train, y_train = train.loc[train_index, features], train.loc[train_index, 'signal']
    X_test, y_test = train.loc[test_index, features], train.loc[test_index, 'signal']

    dtrain = xgb.DMatrix(X_train.values, y_train)
    dtest = xgb.DMatrix(X_test.values, y_test)
    gbm2 = xgb.train(params, dtrain, num_trees)

    XGB3_train[test_index] = gbm2.predict(dtest)
    #predXGB2_test += [gbm2.predict(xgb.DMatrix(test[features].values))]

#XGB3 = sum(predXGB2_test) / len(predXGB2_test)

#test

X_train, y_train = train[features], train['signal']
dtrain = xgb.DMatrix(X_train.values, y_train)
gbm2 = xgb.train(params, dtrain, num_trees)

XGB3_agreement = gbm2.predict(xgb.DMatrix(check_agreement[features].values))
XGB3_correlation = gbm2.predict(xgb.DMatrix(check_correlation[features].values))
XGB3 = gbm2.predict(xgb.DMatrix(test[features].values))

CPU times: user 53min 41s, sys: 1.4 s, total: 53min 42s
Wall time: 1min 25s


In [14]:
print roc_auc_score(train.signal, XGB3_train)
print check_ks(XGB3_agreement)
print compute_cvm(XGB3_correlation, check_correlation['mass'])

0.956325347854
('KS metric', 0.27892192158168361, False)
0.00106445823974


In [15]:
%%time

predXGB0_test = []

params = {"objective": "binary:logistic",
          "eta": 0.5,
          "max_depth": 155,
          "min_child_weight": 0.1,
          "silent": 1,
          "subsample": 1,
          "colsample_bytree": 0.4,
          'eval_metric': 'auc',
          "seed": 4,
          "nthread": 1}
num_trees=3
is_it_wood = True

features_m = features + ['new_mass']

#cv

XGB4_train = np.zeros(len(train))

for train_index, test_index in kf:
    X_train, y_train = train.loc[train_index, features_m], train.loc[train_index, 'signal']
    X_test, y_test = train.loc[test_index, features_m], train.loc[test_index, 'signal']

    dtrain = xgb.DMatrix(X_train.values, y_train)
    dtest = xgb.DMatrix(X_test.values, y_test)
    gbm0 = xgb.train(params, dtrain, num_trees)

    XGB4_train[test_index] = gbm0.predict(dtest)
    #predXGB0_test += [gbm0.predict(xgb.DMatrix(test[features_m].values))]
    
#XGB4 = sum(predXGB0_test) / len(predXGB0_test)

#test

X_train, y_train = train[features_m], train['signal']
dtrain = xgb.DMatrix(X_train.values, y_train)
gbm0 = xgb.train(params, dtrain, num_trees)

XGB4_agreement = gbm0.predict(xgb.DMatrix(check_agreement[features_m].values))
XGB4_correlation = gbm0.predict(xgb.DMatrix(check_correlation[features_m].values))
XGB4 = gbm0.predict(xgb.DMatrix(test[features_m].values))

CPU times: user 14.5 s, sys: 560 ms, total: 15.1 s
Wall time: 15.1 s


In [16]:
print roc_auc_score(train.signal, XGB4_train)
print check_ks(XGB4_agreement)
print compute_cvm(XGB4_correlation, check_correlation['mass'])

0.959014557307
('KS metric', 0.039837807154494531, True)
0.0164632393336


## Let's make 'mass' feature more precise by trying to predict it

In [29]:
%%time

predXGB5 = np.zeros(len(train))
predXGB5_test = []
predXGB5_agreement = []
predXGB5_correlation = []

params = {"objective": "reg:linear",
          "eta": 0.05,
          "max_depth": 6,
          "min_child_weight": 10,
          "silent": 1,
          "subsample": .8,
          "colsample_bytree": 0.9,
          "seed": 1,
          "nthread": nthread}
num_trees=700*2*2

features_m = features + ['new_mass', 'p', 'speed', 'pz']

KF = [KFold(len(train), n_folds=5, random_state=1, shuffle=True),
      KFold(len(train), n_folds=5, random_state=2, shuffle=True),
      KFold(len(train), n_folds=5, random_state=3, shuffle=True),
      KFold(len(train), n_folds=5, random_state=4, shuffle=True),
      KFold(len(train), n_folds=5, random_state=5, shuffle=True),
      KFold(len(train), n_folds=5, random_state=6, shuffle=True),
      KFold(len(train), n_folds=5, random_state=555, shuffle=True)
     ]

for kfi in KF:
    for train_index, test_index in kfi:
        X_train, y_train = train.loc[train_index, features_m], train.loc[train_index, 'mass']
        X_test, y_test = train.loc[test_index, features_m], train.loc[test_index, 'mass']

        dtrain = xgb.DMatrix(X_train.values, y_train, missing=np.nan)
        dtest = xgb.DMatrix(X_test.values, y_test, missing=np.nan)
        gbm5 = xgb.train(params, dtrain, num_trees)

        predXGB5[test_index] += gbm5.predict(dtest)
        predXGB5_test += [gbm5.predict(xgb.DMatrix(test[features_m].values, missing=np.nan))]
        predXGB5_agreement += [gbm5.predict(xgb.DMatrix(check_agreement[features_m].values, missing=np.nan))]
        predXGB5_correlation += [gbm5.predict(xgb.DMatrix(check_correlation[features_m].values, missing=np.nan))]

predXGB5 /= len(KF)
predXGB5_test = sum(predXGB5_test) / len(predXGB5_test)
predXGB5_agreement = sum(predXGB5_agreement) / len(predXGB5_agreement)
predXGB5_correlation = sum(predXGB5_correlation) / len(predXGB5_correlation)

CPU times: user 22h 47min 30s, sys: 28.7 s, total: 22h 47min 58s
Wall time: 36min 11s


In [30]:
train['new_mass2'] = predXGB5
test['new_mass2'] = predXGB5_test
check_agreement['new_mass2'] = predXGB5_agreement
check_correlation['new_mass2'] = predXGB5_correlation

In [31]:
#add two new features

for tt in train, test, check_agreement, check_correlation:
    tt['new_mass_delta'] = tt['new_mass'] - tt['new_mass2']
    tt['new_mass_ratio'] = (1e-10 + tt['new_mass']) / (1e-10 + tt['new_mass2'])

In [32]:
train.to_csv('data/new_train.csv', index=False)
test.to_csv('data/new_test.csv', index=False)
check_agreement.to_csv('data/new_check_agreement.csv', index=False)
check_correlation.to_csv('data/new_check_correlation.csv', index=False)

## train XGB and NN with new 'mass' features

In [18]:
%%time

predXGB8_test = []

params = {"objective": "binary:logistic",
          "eta": 0.05,
          "max_depth": 6,
          "min_child_weight": 10,
          "silent": 1,
          "subsample": 1,
          "colsample_bytree": 1,
          'eval_metric': 'auc',
          "seed": 1,
          "nthread": nthread}
num_trees=1500

features_m = features + ['new_mass', 'new_mass2', 'new_mass_delta', 'new_mass_ratio']

#cv

XGB5_train = np.zeros(len(train))

for train_index, test_index in kf:
    X_train, y_train = train.loc[train_index, features_m], train.loc[train_index, 'signal']
    X_test, y_test = train.loc[test_index, features_m], train.loc[test_index, 'signal']

    dtrain = xgb.DMatrix(X_train.values, y_train)
    dtest = xgb.DMatrix(X_test.values, y_test)
    gbm8 = xgb.train(params, dtrain, num_trees)

    XGB5_train[test_index] = gbm8.predict(dtest)
    #predXGB8_test += [gbm8.predict(xgb.DMatrix(test[features_m].values, missing=np.nan))]

#XGB5 = sum(predXGB8_test) / len(predXGB8_test)

#test

X_train, y_train = train[features_m], train['signal']
dtrain = xgb.DMatrix(X_train.values, y_train)
gbm8 = xgb.train(params, dtrain, num_trees)

XGB5_agreement = gbm8.predict(xgb.DMatrix(check_agreement[features_m].values))
XGB5_correlation = gbm8.predict(xgb.DMatrix(check_correlation[features_m].values))
XGB5 = gbm8.predict(xgb.DMatrix(test[features_m].values))

CPU times: user 1h 47min 59s, sys: 5.19 s, total: 1h 48min 5s
Wall time: 2min 49s


In [19]:
print roc_auc_score(train.signal, XGB5_train)
print check_ks(XGB5_agreement)
print compute_cvm(XGB5_correlation, check_correlation['mass'])

0.998821488766
('KS metric', 0.22408465263773053, False)
0.0813996929384


In [20]:
import nolearn
from lasagne.layers import DenseLayer, InputLayer, DropoutLayer, NINLayer
from lasagne.nonlinearities import softmax, rectify, sigmoid
from lasagne.updates import nesterov_momentum, rmsprop, adagrad, sgd, adadelta
from nolearn.lasagne import NeuralNet
from joblib import Parallel, delayed

def fun(i):
    np.random.seed(i)
    net0 = NeuralNet(layers=layers0,

                     input_shape=(None, Xtrain.shape[1]),
                     dense1_num_units=8,

                     output_num_units=2,
                     output_nonlinearity=softmax,

                     update=adagrad,#
                     update_learning_rate=0.01,

                     eval_size=.005,
                     verbose=1,
                     max_epochs=3000)

    net0.fit(Xtrain, Ytrain.astype(np.int32)) ;
    return net0

In [ ]:
#%%time

import sklearn
NN_train = np.zeros(len(train))
NN_agreement = []
NN_correlation = []
NN = []

features_n = features + ['new_mass', 'new_mass2', 'new_mass_delta', 'new_mass_ratio']

for j, (itrain, itest) in enumerate(kf):
    
    Xtrain, Xtest, Ytrain, Ytest = (train.loc[itrain, features_n], train.loc[itest, features_n], 
                                    train.loc[itrain, 'signal'].values, train.loc[itest, 'signal'].values)
    
    scaler = sklearn.preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    Xbigtest = scaler.transform(test[features_n])
    Xagreement = scaler.transform(check_agreement[features_n])
    Xcorrelation = scaler.transform(check_correlation[features_n])
    
    layers0 = [('input', InputLayer),
               ('dense1', DenseLayer),
               ('output', DenseLayer)]
    
    #this just trains NNs in parallel
    nets = Parallel(n_jobs=-1)(delayed(fun)(c) for c in xrange(10))
    for net in nets:
        NN_train[itest] += net.predict_proba(Xtest)[:,1]
        NN += [net.predict_proba(Xbigtest)[:,1]]
        NN_agreement += [net.predict_proba(Xagreement)[:,1]]
        NN_correlation += [net.predict_proba(Xcorrelation)[:,1]]

NN = sum(NN) / len(NN)
NN_train /= 10
NN_agreement = sum(NN_agreement) / len(NN_agreement)
NN_correlation = sum(NN_correlation) / len(NN_correlation)

In [665]:
print 1

1


In [666]:
print roc_auc_score(train.signal, NN_train)
print check_ks(NN_agreement)
print compute_cvm(NN_correlation, check_correlation['mass'])

0.998404626703
('KS metric', 0.00080544714621538956, True)
0.120509495848


# 4. Combining predictions

In [675]:
# combine estimators in the first part of the solution
v = lambda x: (x[0] ** 0.5 * x[1] ** 2 + x[1] ** 1 * 0) / 2
w = lambda x: (x[0] * x[1] ** .85 * x[2] ** .01 + x[1] * x[3] ** 900 * .85 + x[2] ** 1000 * 2) / 3.85
q = lambda x: x[1] ** 3.9 * .5 + x[0] ** .6 * .2 + x[0] ** .01 * x[1] ** .2 * .0001

vt = [XGB1_train, XGB2_train]
wt = [XGB3_train, XGB4_train, XGB5_train, NN_train]

va = [XGB1_agreement, XGB2_agreement]
wa = [XGB3_agreement, XGB4_agreement, XGB5_agreement, NN_agreement]

vc = [XGB1_correlation, XGB2_correlation]
wc = [XGB3_correlation, XGB4_correlation, XGB5_correlation, NN_correlation]

roc_i = np.array(train.min_ANNmuon > .4)
print evaluation.roc_auc_truncated(train.signal.values[roc_i], (q([v(vt), w(wt)]) / 1)[roc_i])
print check_ks(q([v(va), w(wa)]))
print compute_cvm(q([v(vc), w(wc)]), check_correlation['mass'])

0.999997749873
('KS metric', 0.086290999491298842, True)
0.00137744269512


In [670]:
vtest = [XGB1, XGB2]
wtest = [XGB3, XGB4, XGB5, NN]

submission = pd.DataFrame({"id": test["id"], "prediction": q([v(vtest), w(wtest)])})
submission.to_csv('submit_last.csv', index=False)